In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split,ShuffleSplit,GridSearchCV,cross_val_score
from sklearn.metrics import mean_squared_error,r2_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale
from sklearn import model_selection

In [2]:
hit = pd.read_csv("Hitters.csv")
df = hit.copy()
df = df.dropna()
dms = pd.get_dummies(df[["League","Division","NewLeague"]])
y = df["Salary"]
X_ = df.drop(["Salary","League","Division","NewLeague"],axis = 1).astype("float64")
X = pd.concat([X_,dms[["League_N","Division_W","NewLeague_N"]]],axis = 1)
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.25,random_state = 42)

In [3]:
from sklearn.preprocessing import StandardScaler

In [4]:
scaler = StandardScaler()

In [5]:
scaler.fit(X_train)

StandardScaler()

In [6]:
X_train_scaled = scaler.transform(X_train)

In [7]:
X_test_scaled = scaler.transform(X_test)

In [8]:
from sklearn.neural_network import MLPRegressor

In [9]:
mlp_model = MLPRegressor().fit(X_train_scaled,y_train)

C:\Users\ADMIN\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [10]:
mlp_model

MLPRegressor()

In [11]:
mlp_model.n_layers_

3

In [12]:
mlp_model.hidden_layer_sizes

(100,)

# Tahmin

In [13]:
mlp_model.predict(X_train_scaled)[0:5]

array([10.34884338, 54.48213575, 33.83814108,  8.52314692,  6.23139246])

In [14]:
y_pred = mlp_model.predict(X_test_scaled)

In [15]:
np.sqrt(mean_squared_error(y_test,y_pred))

656.7472572457954

# Model Tuning

In [16]:
mlp_params = {"alpha":[0.1,0.01,0.005],
             "hidden_layer_sizes":[(20,20),(100,50,150),(200,300,150)],
             "activation":["relu","logistic"]}

In [17]:
from sklearn.model_selection import GridSearchCV

In [18]:
mlp_cv_model = GridSearchCV(mlp_model,mlp_params,cv = 10)

In [21]:
import warnings
warnings.filterwarnings('ignore')
mlp_cv_model.fit(X_train_scaled,y_train)

GridSearchCV(cv=10, estimator=MLPRegressor(),
             param_grid={'activation': ['relu', 'logistic'],
                         'alpha': [0.1, 0.01, 0.005],
                         'hidden_layer_sizes': [(20, 20), (100, 50, 150),
                                                (200, 300, 150)]})

In [22]:
mlp_cv_model.best_params_

{'activation': 'relu', 'alpha': 0.1, 'hidden_layer_sizes': (100, 50, 150)}

In [23]:
mlp_tuned = MLPRegressor(alpha = 0.1,hidden_layer_sizes = (100,50,150)) 

In [24]:
mlp_tuned.fit(X_train_scaled,y_train)

MLPRegressor(alpha=0.1, hidden_layer_sizes=(100, 50, 150))

In [25]:
y_pred = mlp_tuned.predict(X_test_scaled)

In [26]:
np.sqrt(mean_squared_error(y_test,y_pred))

353.64767801250423